In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/NNDL/Project/ARDL

/content/drive/MyDrive/Colab Notebooks/NNDL/Project/ARDL


In [3]:
%load_ext autoreload
%autoreload 2

from dataset import *

In [4]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)


In [5]:
emb_mat, data = create_inp_data_and_weights(newsgroups_train, newsgroups_test)

Vocabulary size [Bag-of-words]:  151353
Vocabulary size [GloVe]:  400000


In [7]:
from cnn import *
input_size, emb_size = emb_mat.shape

model = build_random_cnn(3, 8, 128, 256, input_size, emb_size, 7, emb_mat)
model

CNN(
  (embedding): Embedding(151353, 50)
  (conv_layers): Sequential(
    (0): Conv1d(500, 238, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): Dropout(p=0.5, inplace=False)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(238, 173, kernel_size=(5,), stride=(1,), padding=(2,))
    (5): Dropout(p=0.5, inplace=False)
    (6): ReLU()
    (7): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (8): Conv1d(173, 143, kernel_size=(5,), stride=(1,), padding=(2,))
    (9): Dropout(p=0.5, inplace=False)
    (10): ReLU()
    (11): AdaptiveMaxPool1d(output_size=20)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=2860, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=7, bias=True)
)

In [8]:
learning_rate = 0.001
num_epochs = 10
batch_size = 100
display_step = 1


criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

In [9]:
from torch.autograd import Variable

In [10]:
for epoch in range(num_epochs):
    total_batch = int(len(newsgroups_train.data)/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_x,batch_y = get_batch(data['train'],i,batch_size)

        articles = Variable(torch.LongTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        # print("articles",articles)
        # print(batch_x, labels)
        # print("size labels",labels.size())

        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = model(articles)
        #print(outputs.shape, labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 4 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(newsgroups_train.data)//batch_size, loss.data))
        

/content/drive/MyDrive/Colab Notebooks/NNDL/Project/ARDL/cnn.py:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Epoch [1/10], Step [4/113], Loss: 1.8028
Epoch [1/10], Step [8/113], Loss: 1.8194
Epoch [1/10], Step [12/113], Loss: 1.7846
Epoch [1/10], Step [16/113], Loss: 1.8613
Epoch [1/10], Step [20/113], Loss: 1.8211
Epoch [1/10], Step [24/113], Loss: 1.7791
Epoch [1/10], Step [28/113], Loss: 1.7577
Epoch [1/10], Step [32/113], Loss: 1.9051
Epoch [1/10], Step [36/113], Loss: 1.8190
Epoch [1/10], Step [40/113], Loss: 1.7242
Epoch [1/10], Step [44/113], Loss: 1.7168
Epoch [1/10], Step [48/113], Loss: 1.7420
Epoch [1/10], Step [52/113], Loss: 1.6699
Epoch [1/10], Step [56/113], Loss: 1.8062
Epoch [1/10], Step [60/113], Loss: 1.7439
Epoch [1/10], Step [64/113], Loss: 1.6908
Epoch [1/10], Step [68/113], Loss: 1.6388
Epoch [1/10], Step [72/113], Loss: 1.7271
Epoch [1/10], Step [76/113], Loss: 1.6942
Epoch [1/10], Step [80/113], Loss: 1.6672
Epoch [1/10], Step [84/113], Loss: 1.6360
Epoch [1/10], Step [88/113], Loss: 1.7355
Epoch [1/10], Step [92/113], Loss: 1.6352
Epoch [1/10], Step [96/113], Loss: 1

In [11]:
correct = 0
total = 0
total_test_data = len(data['test']['target'])
batch_x_test,batch_y_test = get_batch(data['test'],0,total_test_data)

articles = Variable(torch.LongTensor(batch_x_test))
labels = torch.LongTensor(batch_y_test)
outputs = model(articles)

_, predicted = torch.max(outputs.data, 1)
total += labels.size(0)
correct += (predicted == labels).sum()

print('Accuracy of the network on the 1180 test articles: %d %%' % (100 * correct / total))

Accuracy of the network on the 1180 test articles: 57 %


/content/drive/MyDrive/Colab Notebooks/NNDL/Project/ARDL/cnn.py:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
